## Aaron P Lewis

In [1]:
# Dependencies and Setup
%matplotlib inline
from scipy import stats
from pprint import pprint
import random
import json
import os
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from matplotlib import rcParams
import requests
import numpy as np
import gmaps
# Google developer API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the cities_weather_data.csv file that was create by WeatherPy.
filepath = os.path.join("..", "Resources", "cities_weather_data.csv")

# Read file and store into Pandas DataFrames
cities_weather_data_df = pd.read_csv(filepath)

# Visualize
cities_weather_data_df.head()

,City_id,Country,City_name,Latitude,Longitude,Temperature,Humidity,Clouding,Windspeed,Date
0,6534444,IT,Ossuccio,45.97,9.18,57.00,100,20,2.24,1592541225
1,2944516,DE,Breitenworbis,51.40,10.42,55.00,100,100,3.87,1592541225
2,4011741,MX,Durango,24.83,-104.83,67.19,56,99,7.27,1592541071
3,2977878,FR,Saint-Nicolas-des-Biefs,46.06,3.78,53.60,87,75,2.44,1592541226
4,663128,RO,Vasilaţi,44.29,26.45,63.00,92,100,7.00,1592541226


In [3]:
cities_weather_data_df.loc[cities_weather_data_df['Country'].isnull() == True, : ]

,City_id,Country,City_name,Latitude,Longitude,Temperature,Humidity,Clouding,Windspeed,Date
792,3359638,NaN,Walvis Bay,-22.96,14.51,75.99,26,0,5.99,1592541375


In [4]:
# Sort the DataFrame by Latitude and Longitude.
cities_data_sorted_df = cities_weather_data_df.sort_values(by = ['Latitude', 'Longitude']).reset_index(drop=True)

# Visualize
cities_data_sorted_df.head()

,City_id,Country,City_name,Latitude,Longitude,Temperature,Humidity,Clouding,Windspeed,Date
0,3833094,AR,28 de Noviembre,-51.65,-72.30,28.65,96,37,1.16,1592541336
1,2178971,NZ,Woodlands,-46.37,168.55,39.00,83,0,3.00,1592541296
2,3898880,CL,Balmaceda,-45.92,-71.68,41.00,80,75,6.93,1592541333
3,6206253,NZ,Arrowtown,-44.94,168.81,35.83,82,0,0.27,1592541348
4,2179466,NZ,Westland District,-43.50,170.00,42.71,72,44,4.25,1592541357


In [5]:
# Get a quick summary of DataFrame.
cities_data_sorted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City_id      800 non-null    int64  
 1   Country      799 non-null    object 
 2   City_name    800 non-null    object 
 3   Latitude     800 non-null    float64
 4   Longitude    800 non-null    float64
 5   Temperature  800 non-null    float64
 6   Humidity     800 non-null    int64  
 7   Clouding     800 non-null    int64  
 8   Windspeed    800 non-null    float64
 9   Date         800 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 62.6+ KB


In [6]:
# Locate the missing Country data
cities_data_sorted_df.loc[cities_data_sorted_df['Country'].isnull() == True, : ]

,City_id,Country,City_name,Latitude,Longitude,Temperature,Humidity,Clouding,Windspeed,Date
192,3359638,NaN,Walvis Bay,-22.96,14.51,75.99,26,0,5.99,1592541375


### Humidity Heatmap
* Configure gmaps.
* Use the Latitude and Longitude as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Access maps with unique API key
gmaps.configure(api_key=gkey)

# Store 'Latitude' and 'Longitude' into  locations 
locations = cities_data_sorted_df[["Latitude", "Longitude"]]

In [8]:
locations.head()

,Latitude,Longitude
0,-51.65,-72.30
1,-46.37,168.55
2,-45.92,-71.68
3,-44.94,168.81
4,-43.50,170.00


In [9]:
# Create a Humidity Heatmap layer
humidity = cities_data_sorted_df["Humidity"]

# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# choice of map_type="HYBRID", "TERRAIN" 
fig = gmaps.figure(layout = figure_layout, center=(9.0, 45.0), zoom_level=2, map_type = "HYBRID")

humidity_heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(humidity_heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    - A maximun temperature higher than 70 but lower than 80 degrees.
    - Wind speed less than 10 mph.
    - Zero cloudiness.
* Drop any rows will null values.

In [10]:
# Vacation conditions.
temp = (cities_data_sorted_df["Temperature"] > 70) & (cities_data_sorted_df["Temperature"] < 80)
wind = cities_data_sorted_df["Windspeed"] < 10
cloud = cities_data_sorted_df["Clouding"] == 0

vacation_spot_df = cities_data_sorted_df[(temp) & (wind) & (cloud)]
vacation_spot_df

,City_id,Country,City_name,Latitude,Longitude,Temperature,Humidity,Clouding,Windspeed,Date
192,3359638,NaN,Walvis Bay,-22.96,14.51,75.99,26,0,5.99,1592541375
264,3388145,BR,São Raimundo Nonato,-9.02,-42.70,70.09,78,0,4.56,1592541304
309,3398007,BR,Isaías Coelho,-7.74,-41.68,71.51,82,0,7.11,1592541336
463,88834,LY,Tūkrah,32.53,20.58,77.94,35,0,1.34,1592541272
486,259819,GR,Nomós Kykládon,37.42,24.92,78.01,57,0,3.36,1592541287
674,608073,KZ,Taskopa,48.85,55.58,76.55,29,0,8.32,1592541232


In [11]:
# Quick summary of Vacation DataFrame.
vacation_spot_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 192 to 674
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City_id      6 non-null      int64  
 1   Country      5 non-null      object 
 2   City_name    6 non-null      object 
 3   Latitude     6 non-null      float64
 4   Longitude    6 non-null      float64
 5   Temperature  6 non-null      float64
 6   Humidity     6 non-null      int64  
 7   Clouding     6 non-null      int64  
 8   Windspeed    6 non-null      float64
 9   Date         6 non-null      int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 528.0+ bytes


### Comfortable Humidity Level
* Relative humidity less than or equal to 55%: dry and comfortable.
* Between 55 and 65: becoming "sticky" with muggy in evenings.
* Greater than or equal to 65: lots of moisture in the air, becoming oppressive.

In [12]:
comfortable_df = vacation_spot_df[vacation_spot_df["Humidity"] < 55].reset_index()
comfortable_df

,index,City_id,Country,City_name,Latitude,Longitude,Temperature,Humidity,Clouding,Windspeed,Date
0,192,3359638,NaN,Walvis Bay,-22.96,14.51,75.99,26,0,5.99,1592541375
1,463,88834,LY,Tūkrah,32.53,20.58,77.94,35,0,1.34,1592541272
2,674,608073,KZ,Taskopa,48.85,55.58,76.55,29,0,8.32,1592541232


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# Find the closest hotel to each of the coordinates
target_type = "lodging"
target_search = "hotel"
radius = 5000

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Use iterrows to iterate through comfortable_df dataframe.
# Tuple unpacking
# Parallel assignment
for index, row in comfortable_df.iterrows():
    
    # params dict
    lat = row["Latitude"]
    long = row["Longitude"]
    
    params = {"location": f"{lat},{long}",
              "types": target_type,
              "keyword": target_search,
              "radius": radius,
              "key": gkey
             }

    # Get City,Country from df
    city = row['City_name']
    country = row['Country']

    # Assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}, {country}.")
    response = requests.get(base_url, params=params).json()
    
    #print(response.url)
    
    # extract results
    results = response['results']
    
    try:        
        comfortable_df.loc[index, 'Hotel'] = results[0]['name']

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Index 0: Walvis Bay, nan.
Retrieving Results for Index 1: Tūkrah, LY.
Retrieving Results for Index 2: Taskopa, KZ.
Missing field/result... skipping.


In [14]:
hotel_df = comfortable_df.copy()
hotel_df

,index,City_id,Country,City_name,Latitude,Longitude,Temperature,Humidity,Clouding,Windspeed,Date,Hotel
0,192,3359638,NaN,Walvis Bay,-22.96,14.51,75.99,26,0,5.99,1592541375,Protea Hotel by Marriott Walvis Bay Pelican Bay
1,463,88834,LY,Tūkrah,32.53,20.58,77.94,35,0,1.34,1592541272,قاعدة فيقا فيقا
2,674,608073,KZ,Taskopa,48.85,55.58,76.55,29,0,8.32,1592541232,NaN


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City_name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [16]:
# Create a map using city centroid coordinates to set markers
marker_locations = hotel_df[["Latitude", "Longitude"]]

# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#fig = gmaps.figure(layout = figure_layout, map_type = "HYBRID")
# choice of map_type="HYBRID", "TERRAIN" 
fig = gmaps.figure(layout = figure_layout, center=(9.0, 45.0), zoom_level=2, map_type = "HYBRID")

# Create a marker_layer using the hotel list to fill the info box
hotel_markers = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)
fig.add_layer(hotel_markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [17]:
# Add marker layer ontop of heat map
# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#fig = gmaps.figure(layout = figure_layout, map_type = "HYBRID")
# choice of map_type="HYBRID", "TERRAIN" 
fig = gmaps.figure(layout = figure_layout, center=(9.0, 45.0), zoom_level=2, map_type = "HYBRID")

fig.add_layer(humidity_heat_layer)
fig.add_layer(hotel_markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…